<a href="https://colab.research.google.com/github/AntonShih/LoRA_TinyLlama/blob/master/unsloth2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 8888,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("AntonShih/FineTome_100k_basic_knowledge_of_mold", split = "train")

(…)neTome_100k_basic_knowledge_of_mold.json:   0%|          | 0.00/67.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
dataset[5]["conversations"]

[{'content': '黴菌是否能在極端乾燥的環境中存活？', 'role': 'user'},
 {'content': '黴菌在極端乾燥環境中無法活躍生長，但能以孢子形式存活很長時間。這些孢子具有較強的抵抗力，可在乾燥環境中保持休眠狀態數月甚至數年。一旦環境條件恢復適宜（濕度增加），孢子就會重新活化並開始生長繁殖。這就是為什麼即使完全乾燥的鞋子，在遇到潮濕環境後仍可能發霉。',
  'role': 'assistant'}]

In [8]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n黴菌是否能在極端乾燥的環境中存活？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n黴菌在極端乾燥環境中無法活躍生長，但能以孢子形式存活很長時間。這些孢子具有較強的抵抗力，可在乾燥環境中保持休眠狀態數月甚至數年。一旦環境條件恢復適宜（濕度增加），孢子就會重新活化並開始生長繁殖。這就是為什麼即使完全乾燥的鞋子，在遇到潮濕環境後仍可能發霉。<|eot_id|>'

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
# from transformers import EarlyStoppingCallback

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
            per_device_train_batch_size = 8,  # 讓模型學更多
            gradient_accumulation_steps = 2,  # 更頻繁更新權重
            num_train_epochs = 13,  # 讓數據學 13 次
            max_steps = -1,  # 讓它依照 epochs 計算步數
            learning_rate = 1.5e-5,  # 降低學習率，讓學習更細緻
            lr_scheduler_type = "cosine_with_restarts",  # 讓學習率下降更平滑
            warmup_steps = 5,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 1,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            seed = 8888,
            output_dir = "outputs",
            report_to = "none",
            # save_strategy = "epoch",  # 每個 epoch 儲存 checkpoint
            # evaluation_strategy = "epoch",  # 每個 epoch 評估
            # load_best_model_at_end = True,  # 訓練結束時載入最佳模型
            # metric_for_best_model = "loss",  # 監控 loss 來決定最佳模型
        ),
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # 如果 3 個 epoch 沒有進步，
    )

Unsloth: We found double BOS tokens - we shall remove one automatically.


In [18]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",)

In [19]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n黴菌是否能在極端乾燥的環境中存活？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n黴菌在極端乾燥環境中無法活躍生長，但能以孢子形式存活很長時間。這些孢子具有較強的抵抗力，可在乾燥環境中保持休眠狀態數月甚至數年。一旦環境條件恢復適宜（濕度增加），孢子就會重新活化並開始生長繁殖。這就是為什麼即使完全乾燥的鞋子，在遇到潮濕環境後仍可能發霉。<|eot_id|>'

In [20]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                   \n\n黴菌在極端乾燥環境中無法活躍生長，但能以孢子形式存活很長時間。這些孢子具有較強的抵抗力，可在乾燥環境中保持休眠狀態數月甚至數年。一旦環境條件恢復適宜（濕度增加），孢子就會重新活化並開始生長繁殖。這就是為什麼即使完全乾燥的鞋子，在遇到潮濕環境後仍可能發霉。<|eot_id|>'

In [21]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.32 GB of memory reserved.


In [22]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 13 | Total steps = 78
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.725900
2,1.790200
3,1.782600
4,1.744000
5,1.808400
6,1.760800
7,2.005000
8,1.786300
9,1.701600
10,1.766300


In [ ]:
print(f"Current steps completed: {trainer.state.global_step}")

Current steps completed: 78


In [23]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

304.1023 seconds used for training.
5.07 minutes used for training.
Peak reserved memory = 3.953 GB.
Peak reserved memory for training = 1.633 GB.
Peak reserved memory % of max memory = 26.816 %.
Peak reserved memory for training % of max memory = 11.078 %.


In [48]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "鞋子上的黴菌是否可能導致腳氣？"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True,
                         temperature = 0.4, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n鞋子上的黴菌是否可能導致腳氣？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n黴菌感染可以導致腳氣。黴菌是一種微生物，通常存在於土壤、水和環境中。當黴菌進入皮膚時，可能會引起感染。腳氣是由黴菌感染導致的皮膚感染，通常出現在腳部。黴菌感染可以導致腳部發炎、腫塊、疼痛和皮膚變色等症狀。<|eot_id|>']

In [ ]:
from google.colab import userdata

model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

# 從 Colab 密碼管理器獲取 token
hf_token = userdata.get('hugging_face_token')

model.push_to_hub("AntonShih/model", token = hf_token) # Online saving
tokenizer.push_to_hub("AntonShih/model", token = hf_token) # Online saving

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/AntonShih/model


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]